In [101]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import  KFold
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pickle
  

In [102]:
# 512 dimenções 
# 256 dimenções 
# 128 dimenções 
# 64 dimenções 
# 32 dimenções 
# 16 dimenções 
DATASET_DIR = "modificado_result_final_resnet101.csv"
N_COMPONENTS = 512

In [103]:
df_resnet101 = pd.read_csv("modificado_result_final_resnet101.csv")
labels_resnet101 = df_resnet101["Class"].to_numpy()
df_resnet101.drop(columns=["Class"], inplace=True)
df_resnet101.drop(columns=["Filename"], inplace=True)
df_resnet101_numpy = df_resnet101.to_numpy()

In [104]:
df_resnet101

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_2038,feature_2039,feature_2040,feature_2041,feature_2042,feature_2043,feature_2044,feature_2045,feature_2046,feature_2047
0,0.000000,0.000000,0.013706,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001027,...,0.000000,0.000000,0.000000,0.016515,0.003167,0.000856,0.000239,0.021120,0.000000,0.000000
1,0.035060,0.000000,0.000000,0.022573,0.006494,0.005687,0.009803,0.000000,0.030033,0.000000,...,0.035230,0.008297,0.000000,0.051527,0.000000,0.000000,0.053514,0.000000,0.003269,0.000000
2,0.028943,0.000000,0.017708,0.008827,0.070061,0.107132,0.008053,0.000000,0.021134,0.006830,...,0.052321,0.004084,0.000000,0.007507,0.000000,0.008239,0.038493,0.054490,0.232083,0.000000
3,0.085944,0.012458,0.000000,0.000000,0.160621,0.008041,0.111115,0.000000,0.154407,0.071641,...,0.034166,0.014321,0.006400,0.095789,0.011407,0.115172,0.000000,0.000526,0.002603,0.014679
4,0.196794,0.000000,0.021515,0.043528,0.000000,0.000000,0.010194,0.000000,0.001453,0.044624,...,0.034795,0.006506,0.000000,0.147823,0.002599,0.000000,0.000359,0.021721,0.035856,0.001480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.034932,0.000000,0.012726,0.001952,0.150972,0.060082,0.001651,0.000000,0.000771,0.050189,...,0.000000,0.000000,0.000291,0.000000,0.000000,0.028684,0.006766,0.092402,0.000000,0.083792
996,0.000000,0.000000,0.012418,0.000000,0.000000,0.065996,0.079543,0.000000,0.002165,0.002406,...,0.003913,0.003491,0.002513,0.144734,0.019566,0.274534,0.004362,0.043479,0.000000,0.000000
997,0.134238,0.002916,0.000000,0.000457,0.002975,0.044325,0.025243,0.000000,0.000000,0.019771,...,0.005641,0.000000,0.000000,0.060947,0.000000,0.141522,0.044818,0.007452,0.000000,0.067198
998,0.006072,0.000579,0.000000,0.143059,0.000000,0.227858,0.004741,0.003457,0.020548,0.050012,...,0.000000,0.001952,0.000000,0.028460,0.002824,0.255584,0.066837,0.001513,0.000000,0.001555


In [105]:
X_train,X_test, y_train, y_test = train_test_split(df_resnet101_numpy, 
                                                   labels_resnet101, 
                                                   test_size=0.2, 
                                                   random_state=42, 
                                                   stratify=labels_resnet101)



In [ ]:
standard_scaler=StandardScaler()
X_train=standard_scaler.fit_transform(X_train)
X_test=standard_scaler.transform(X_test)


pca=PCA(n_components=N_COMPONENTS)
X_train=pca.fit_transform(X_train)
X_test=pca.transform(X_test)

In [107]:
# KNN
pipeline=Pipeline([
    ('knn', KNeighborsClassifier())
])

knn_param_grid = {
    "knn__n_neighbors": [3, 5, 7, 9, 11, 13, 15],
    "knn__weights": ['uniform', 'distance'],
    "knn__metric": ['euclidean', 'manhattan'],
    "knn__leaf_size": [20, 30, 40, 50]
}

knn_random_search = GridSearchCV(
    param_grid=knn_param_grid,
    estimator=pipeline,
    cv=KFold(n_splits=10, shuffle=True, random_state=42),
    scoring='accuracy',
)

knn_random_search.fit(X_train, y_train)

print("Best KNN parameters:", knn_random_search.best_params_)
print("Best KNN Scores:", knn_random_search.best_score_) 

/home/galmeidalopes/.local/lib/python3.13/site-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/galmeidalopes/.local/lib/python3.13/site-packages/sklearn/model_selection/_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/home/galmeidalopes/.local/lib/python3.13/site-packages/sklearn/metrics/_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/galmeidalopes/.local/lib/python3.13/site-packages/sklearn/metrics/_scorer.py", line 380, in _score
    y_pred = method_caller(
        estimator,
    ...<2 lines>...
        pos_label=pos_label,
    )
  File "/home/galmeidalopes/.local/lib/python3.13/site-pac

Best KNN parameters: {'knn__leaf_size': 20, 'knn__metric': 'euclidean', 'knn__n_neighbors': 5, 'knn__weights': 'distance'}
Best KNN Scores: 0.7350000000000001


/home/galmeidalopes/.local/lib/python3.13/site-packages/sklearn/model_selection/_search.py:1107: UserWarning: One or more of the test scores are non-finite: [0.69    0.72125 0.7     0.735   0.68875 0.71875 0.695   0.725   0.69375
 0.725   0.6925  0.71125 0.69    0.70625     nan 0.60375     nan 0.575
     nan 0.5625      nan 0.54625     nan 0.52875     nan 0.52        nan
 0.51875 0.69    0.72125 0.7     0.735   0.68875 0.71875 0.695   0.725
 0.69375 0.725   0.6925  0.71125 0.69    0.70625     nan 0.60375     nan
 0.575       nan 0.5625      nan 0.54625     nan 0.52875     nan 0.52
     nan 0.51875 0.69    0.72125 0.7     0.735   0.68875 0.71875 0.695
 0.725   0.69375 0.725   0.6925  0.71125 0.69    0.70625     nan 0.60375
     nan 0.575       nan 0.5625      nan 0.54625     nan 0.52875     nan
 0.52        nan 0.51875 0.69    0.72125 0.7     0.735   0.68875 0.71875
 0.695   0.725   0.69375 0.725   0.6925  0.71125 0.69    0.70625     nan
 0.60375     nan 0.575       nan 0.5625      nan 

In [108]:
print("KNN Test set score: {:.2f}".format(knn_random_search.score(X_test, y_test)))

KNN Test set score: 0.72


In [109]:
# Random Forest
pipeline=Pipeline([
    ('rf', RandomForestClassifier(random_state=42))
])
rf_param_grid = {
    "rf__n_estimators": [100],
    "rf__max_depth": [10,20],
    "rf__min_samples_split": [2, 5,7,8],
}

rf_random_search = GridSearchCV(
    param_grid=rf_param_grid,
    estimator=pipeline,
    cv=KFold(n_splits=10, shuffle=True, random_state=42),
    n_jobs=8,
    scoring='accuracy',
)

rf_random_search.fit(X_train, y_train)

print("Best RF parameters:", rf_random_search.best_params_)
print("Best RF Scores:", rf_random_search.best_score_)


Exception ignored in: <function ResourceTracker.__del__ at 0x7f6bc337ad40>
Traceback (most recent call last):
  File "/usr/lib64/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/usr/lib64/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/usr/lib64/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x7f6d11e76d40>
Traceback (most recent call last):
  File "/usr/lib64/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/usr/lib64/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/usr/lib64/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x7f73bd276d40>
Traceback (most recent call last):
  File "/usr/lib64/python3.13/multip

Best RF parameters: {'rf__max_depth': 10, 'rf__min_samples_split': 7, 'rf__n_estimators': 100}
Best RF Scores: 0.7474999999999999


In [110]:
print("Random Forest Test set score: {:.2f}".format(rf_random_search.score(X_test, y_test)))

Random Forest Test set score: 0.72


In [111]:
# Criação do pipeline
pipeline = Pipeline([
    ('svm', SVC(random_state=42))
])

# Grade de parâmetros — o nome precisa seguir o formato "nome_no_pipeline__parametro"
svm_param_grid = {
    'svm__C': [0.1, 1, 10, 100],
    'svm__kernel': ['linear', 'rbf', 'poly'],
    'svm__gamma': ['scale', 'auto']
}

# Configuração do GridSearchCV
svm_random_search = GridSearchCV(
    estimator=pipeline,
    param_grid=svm_param_grid,
    cv=KFold(n_splits=10, shuffle=True, random_state=42),
    scoring='accuracy',
    n_jobs=-1  # usa todos os núcleos disponíveis
)

# Treinamento do GridSearch
svm_random_search.fit(X_train, y_train)

# Exibir os melhores parâmetros e pontuação
print("Best SVM parameters:", svm_random_search.best_params_)
print("Best SVM Score:", svm_random_search.best_score_)

Best SVM parameters: {'svm__C': 10, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}
Best SVM Score: 0.8450000000000001


In [112]:
print("SVM Test set score: {:.2f}".format(svm_random_search.score(X_test, y_test)))

SVM Test set score: 0.84


In [113]:
# Criação do pipeline
pipeline = Pipeline([
    ('mlp', MLPClassifier(random_state=42, max_iter=1000))
])

# Grade de parâmetros — agora para MLP
mlp_param_grid = {
    'mlp__hidden_layer_sizes': [(50,),(100,),(50,100) ,(100, 50)],
    'mlp__activation': ['relu', 'tanh'],
    'mlp__solver': ['adam', 'lbfgs'],
    'mlp__alpha': [0.0001, 0.001, 0.01],
}

# Configuração do GridSearchCV
mlp_grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=mlp_param_grid,
    cv=KFold(n_splits=10, shuffle=True, random_state=42),
    scoring='accuracy',
    n_jobs=-1
)

# Treinamento do GridSearch
mlp_grid_search.fit(X_train, y_train)

# Exibir os melhores parâmetros e pontuação
print("Best MLP parameters:", mlp_grid_search.best_params_)
print("Best MLP Score:", mlp_grid_search.best_score_)


Best MLP parameters: {'mlp__activation': 'tanh', 'mlp__alpha': 0.0001, 'mlp__hidden_layer_sizes': (100,), 'mlp__solver': 'lbfgs'}
Best MLP Score: 0.835


In [114]:
print("MLP Test set score: {:.2f}".format(mlp_grid_search.score(X_test, y_test)))

MLP Test set score: 0.84


In [115]:
model = VotingClassifier([
    ("dt", DecisionTreeClassifier(max_depth=4)),
    ("knn", KNeighborsClassifier()),
    ("svm", SVC(probability=True)),
    ("mlp", MLPClassifier(
        hidden_layer_sizes=(100, 50),
        activation="tanh",
        solver="adam",
        learning_rate_init=0.01
    )),
    ("rf", RandomForestClassifier(
        max_depth=20,
        min_samples_split=5,
        n_estimators=100
    )),
],
    weights=[1, 1, 2, 2, 1],
    n_jobs=-1,
    voting="soft"
)


In [116]:

model.fit(X_train, y_train)
probs = np.array([clf.predict_proba(X_test) for clf in model.estimators_])
weights = np.array([1, 1, 2, 2, 1]).reshape(-1, 1, 1)
prod_probs = np.prod(np.power(probs, weights), axis=0)
prod_probs = prod_probs / prod_probs.sum(axis=1, keepdims=True)
y_pred_prod = np.argmax(prod_probs, axis=1)
print("🔸 Classes previstas (regra do produto):")
print(y_pred_prod)

🔸 Classes previstas (regra do produto):
[1 0 2 0 5 2 9 2 4 5 0 0 8 0 9 3 7 5 0 9 2 6 0 7 0 1 9 8 5 5 2 2 5 6 1 6 9
 4 8 3 0 9 7 0 5 5 6 1 4 2 4 6 4 4 8 3 3 1 1 1 6 8 8 9 3 7 9 1 9 9 6 8 1 7
 8 4 0 7 7 7 3 7 8 6 5 9 2 4 9 5 2 8 6 0 2 5 5 2 8 8 6 5 8 7 2 9 3 0 6 3 0
 0 1 2 1 2 1 7 8 7 3 0 5 9 2 1 6 4 6 3 6 7 7 5 7 5 3 1 3 8 1 0 3 0 2 4 4 0
 1 4 7 1 1 0 9 0 4 5 1 0 3 4 4 5 8 9 9 2 7 0 6 5 0 9 7 1 0 4 2 0 6 4 9 4 1
 7 6 4 3 7 4 1 7 7 4 0 2 1 4 1]


/tmp/ipykernel_7967/1974342990.py:5: RuntimeWarning: invalid value encountered in divide
  prod_probs = prod_probs / prod_probs.sum(axis=1, keepdims=True)


In [117]:

y_pred = model.predict(X_test)
print(y_pred)


['foliage' 'fabric' 'plastic' 'fabric' 'paper' 'glass' 'wood' 'glass'
 'metal' 'paper' 'fabric' 'fabric' 'water' 'fabric' 'wood' 'leather'
 'stone' 'paper' 'fabric' 'wood' 'glass' 'plastic' 'fabric' 'stone'
 'fabric' 'foliage' 'wood' 'water' 'paper' 'paper' 'water' 'glass' 'paper'
 'plastic' 'foliage' 'plastic' 'wood' 'metal' 'water' 'leather' 'water'
 'wood' 'stone' 'fabric' 'paper' 'paper' 'plastic' 'foliage' 'metal'
 'glass' 'metal' 'plastic' 'metal' 'metal' 'water' 'leather' 'leather'
 'foliage' 'foliage' 'foliage' 'plastic' 'water' 'water' 'wood' 'leather'
 'stone' 'wood' 'foliage' 'wood' 'wood' 'plastic' 'water' 'foliage'
 'stone' 'water' 'metal' 'fabric' 'stone' 'stone' 'stone' 'leather'
 'stone' 'water' 'plastic' 'leather' 'wood' 'glass' 'metal' 'wood' 'paper'
 'glass' 'water' 'plastic' 'fabric' 'glass' 'paper' 'paper' 'glass'
 'water' 'water' 'plastic' 'paper' 'water' 'stone' 'plastic' 'wood'
 'leather' 'fabric' 'plastic' 'leather' 'fabric' 'fabric' 'foliage'
 'glass' 'foliage

In [118]:
print("Voting Test set score: {:.2f}".format(model.score(X_test, y_test)))

Voting Test set score: 0.83


In [ ]:
knn_random_search.fit(X_test, y_test)
svm_random_search.fit(X_test, y_test)
rf_random_search.fit(X_test, y_test)
mlp_grid_search.fit(X_test, y_test)

with open(f'ensemble_model_pca_{N_COMPONENTS}_resnet101.pkl', 'wb') as f:
    pickle.dump(
        {'knn': knn_random_search,
        'svm': svm_random_search,
        'rf': rf_random_search,
        'mlp': mlp_grid_search,
        'voting': model,
        'scaler': standard_scaler,
        'pca': pca}, f)